In [3]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [4]:
import sys
sys.path.append('..')

from tools.graph_builder import graph_build

In [5]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-ada-002',
    openai_api_version='2023-05-15'
)
graph_store = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='Chunk', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])

In [6]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import AzureChatOpenAI
import os
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)
graph.refresh_schema()
graph_tradition_chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True)
graph_tradition_tool = graph_tradition_chain.as_tool()
graph_tradition_tool


/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


StructuredTool(name='GraphCypherQAChain', description="Takes {'title': 'ChainInput', 'type': 'object', 'properties': {'query': {'title': 'Query'}}}.", args_schema=<class 'pydantic.v1.main.ChainInput'>, func=<function convert_runnable_to_tool.<locals>.invoke_wrapper at 0x142a0c400>, coroutine=<function convert_runnable_to_tool.<locals>.ainvoke_wrapper at 0x142a0c540>)

In [23]:
vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='Chunk', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9}
)
retriever

VectorStoreRetriever(tags=['Neo4jVector', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.neo4j_vector.Neo4jVector object at 0x1479dfd70>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9})

In [24]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [26]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    "你是一個有用的助手, 你的任務是回答問題."
    "你必須根據以下提供的檢索內容及資料庫查詢結果進行問答問題."
    "如果檢索內容為空, 則回答 '沒有找到相關資訊'"
    "以 5 至 10 句話以內回應, 保持答案的簡潔"
    "以下為檢索內容:\n\n"
    "{context}"

    "以下為資料庫查詢結果:\n\n"
    "{tradition_result}"

    問題: {question}
    """
)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "tradition_result": graph_tradition_chain}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
question = "未滿15歲被保險人死亡，身故保險金該怎麼賠?"

In [28]:
retriever.invoke(question)

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/langchain_core/vectorstores/base.py:571: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.9
  warnings.warn(


[]

In [29]:
graph_tradition_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)
RETURN count(d) AS total_documents

Full Context:
[{'total_documents': 5}]

> Finished chain.


{'query': '總共有幾個產品(Document)?', 'result': '總共有5個產品(Document)。'}

In [36]:
rag_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)-[:PART]->(c:Chunk)
WHERE c.content CONTAINS '未滿15歲死亡，身故保險金怎麼賠？'
RETURN d.filename, d.path, c.page_num, c.content

Full Context:
[]

> Finished chain.


'根據提供的檢索內容，如果被保險人未滿15歲死亡，身故保險金會變更為喪葬費用保險金。具體的給付方式如下：\n\n1. 若保險契約於民國九十九年二月三日（不含）前訂立，且喪葬費用保險金額大於或等於遺產及贈與稅法第十七條有關喪葬費扣除額之半數，其喪葬費用保險金的給付依契約約定。\n2. 若喪葬費用保險金額小於上述扣除額之半數，則應加計民國一百零九年六月十二日（含）以後所投保之喪葬費用保險金額，受益人得領取之喪葬費用保險金總和。\n\n因此，未滿15歲死亡的被保險人，主要是按照喪葬費用保險金的相關規定進行賠付。'